In [ ]:
!pip install -U google-genai

In [2]:
import json
import time
import pandas as pd
from google import genai
from google.genai import types
from google.colab import userdata, drive

# 1. Bağlantılar ve Yapılandırma
drive.mount('/content/drive')

# Google API Key'i Colab Secrets'tan alıyoruz
GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GEMINI_API_KEY)

# --- 2025 GÜNCEL MODEL SEÇİMLERİ ---
# Test edilecek model (Cevap veren): En hızlı ve güncel
TEST_MODEL_ID = "gemini-2.0-flash"
# Hakem model (Puanlayan): En güçlü akıl yürütme kapasitesi
JUDGE_MODEL_ID = "gemini-2.0-flash" # Veya varsa "gemini-1.5-pro"

# 2. Hakem (Judge) Fonksiyonu
def judge_with_llm(question, agent_answer, expected_keywords):
    judge_prompt = f"""
      You are an expert Gaming QA Auditor. Evaluate the response accuracy.

      User Input: "{question}"
      Expected Keywords: {expected_keywords}
      Agent's Response: "{agent_answer}"

      Rules:
      - Score 100 if all facts are correct.
      - Return ONLY valid JSON: {{ "accuracy": <0-100>, "reasoning": "..." }}
    """
    try:
        # Yeni SDK'da JSON formatı yapılandırması
        response = client.models.generate_content(
            model=JUDGE_MODEL_ID,
            contents=judge_prompt,
            config=types.GenerateContentConfig(
                response_mime_type='application/json'
            )
        )
        # Yeni SDK'da response.parsed doğrudan JSON objesi dönebilir veya response.text kullanılır
        res_data = json.loads(response.text)
        return max(0, min(100, int(res_data.get("accuracy", 0))))
    except Exception as e:
        print(f"⚠️ Hakem Hatası: {e}")
        return 0

# 3. Ana Benchmark Fonksiyonu
def run_final_benchmark():
    try:
        with open('/content/drive/MyDrive/benchmark/benchmark_questions.json', 'r') as f:
            tasks = json.load(f)
    except FileNotFoundError:
        print("❌ Hata: benchmark_questions.json bulunamadı!")
        return

    results = []
    print(f"🚀 MODERN GEMINI BENCHMARK BAŞLADI")
    print(f"🧪 Test Edilen Model: {TEST_MODEL_ID}")
    print(f"⚖️ Hakem Model: {JUDGE_MODEL_ID}")
    print("="*80)

    for task in tasks:
        print(f"🆔 ID: {task['id']}")
        print(f"❓ SORU: {task['q']}")

        start_time = time.time()

        try:
            # Yanıt Üretimi - Yeni SDK Sözdizimi
            response = client.models.generate_content(
                model=TEST_MODEL_ID,
                contents=task['q'],
                config=types.GenerateContentConfig(
                    system_instruction="You are a professional gaming expert and hardware consultant.",
                    temperature=0.6
                )
            )

            clean_answer = response.text.strip()
            time.sleep(2) # Yeni modellerde RPM limitleri daha esnek olabilir ama güvenli kalalım

        except Exception as e:
            print(f"❌ API Hatası: {e}")
            clean_answer = f"Error: {str(e)}"
            time.sleep(5)

        # Süre ve Puanlama
        duration = round(time.time() - start_time, 2)
        score = judge_with_llm(task['q'], clean_answer, task.get('expected', []))

        print(f"💡 CEVAP: {clean_answer[:100]}...")
        print(f"🎯 SKOR: %{score} | ⏱️ SÜRE: {duration} sn")
        print("-" * 60)

        results.append({
            "ID": task['id'], "Question": task['q'], "Answer": clean_answer,
            "Score": score, "Latency_Sec": duration,
            "Test_Model": TEST_MODEL_ID, "Judge_Model": JUDGE_MODEL_ID
        })

    # 4. Kaydetme
    df = pd.DataFrame(results)
    output_path = '/content/drive/MyDrive/benchmark/gemini_2_report.csv'
    df.to_csv(output_path, index=False)

    print(f"\n✅ TAMAMLANDI | Ortalama Başarı: %{round(df['Score'].mean(), 2)}")
    print(f"📁 Rapor: {output_path}")

# Başlat
run_final_benchmark()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🚀 MODERN GEMINI BENCHMARK BAŞLADI
🧪 Test Edilen Model: gemini-2.0-flash
⚖️ Hakem Model: gemini-2.0-flash
🆔 ID: 1
❓ SORU: What are the main similarities between Skyrim and The Witcher 3? What would you recommend to someone who has played both?
💡 CEVAP: Alright, let's dive into the similarities between Skyrim and The Witcher 3, and then I'll give you s...
🎯 SKOR: %100 | ⏱️ SÜRE: 8.31 sn
------------------------------------------------------------
🆔 ID: 2
❓ SORU: How does the main mechanic (use of light) in Alan Wake integrate with the story?
💡 CEVAP: Alright, let's delve into how the core mechanic of light in Alan Wake is inextricably linked to the ...
🎯 SKOR: %100 | ⏱️ SÜRE: 7.87 sn
------------------------------------------------------------
🆔 ID: 3
❓ SORU: I have a GTX 1650 GPU, can I play GTA V at 1080p high settings?
💡 CEVAP: Alright, let's break down whet